In [1]:
from ultralytics import YOLO
import torch
import numpy as np

In [2]:
# Load the model
model = YOLO("./model/best.pt")

In [3]:
# Images to predict
images = ['./test-images/Brussel-4-crop.jpg']

In [4]:
# Predict the model
results = model(images)


0: 544x640 21 flats, 25 slopeds, 1113.5ms
Speed: 12.7ms preprocess, 1113.5ms inference, 1286.1ms postprocess per image at shape (1, 3, 544, 640)


In [5]:
# Initialize area sums
flat_area = 0  
sloped_area = 0  

# Process results from the model
for result in results:
    boxes = result.boxes  
    masks = result.masks  
    label_names = result.names  

    # Iterate through detected boxes and sum mask areas
    for idx, box in enumerate(boxes):
        box_data = box.data.numpy()  
        label_idx = int(box_data[0, 5])  # Access the class label index
        label = label_names[label_idx]
        
        mask = masks[idx]  
        
        # Access the data attribute of the mask and ensure it is converted to numpy array
        mask_data = mask.data 
        if isinstance(mask_data, torch.Tensor):
            mask_data = mask_data.squeeze(0).numpy()  

        # Calculate area of the mask: sum of all 1's in the mask array
        mask_area = np.sum(mask_data)

        # Increment area sum based on roof type
        if label == 'flat':
            flat_area += mask_area
        elif label == 'sloped':
            sloped_area += mask_area

# Calculate the ratio of areas
if sloped_area > 0:  
    ratio_area_flat_to_steep = flat_area / sloped_area
else:
    ratio_area_flat_to_steep = None

print(f"Total mask area of flat roofs: {flat_area}")
print(f"Total mask area of steep roofs: {sloped_area}")
print(f"Ratio of mask area (flat to steep roofs): {ratio_area_flat_to_steep}")

Total mask area of flat roofs: 195565.0
Total mask area of steep roofs: 11691.0
Ratio of mask area (flat to steep roofs): 16.727824822513043


In [7]:

# Images to predict
images = ['./test-images/Brussel-4-crop.jpg']

# Predict with the model
results = model(images)

# Access the first Results object from the list
first_result = results[0]

# Retrieve label names and boxes from the first result
label_names = first_result.names
boxes = first_result.boxes

# Initialize counts
flat_count = 0
sloped_count = 0

# Iterate through detected boxes and count
for box in boxes:
    # Access the data tensor for this box
    box_data = box.data.numpy()  # Ensure conversion to numpy array if not already
    label_idx = int(box_data[0, 5])  # Access the class label index which is at the 6th position in the data array
    label = label_names[label_idx]
    if label == 'flat':
        flat_count += 1
    elif label == 'sloped':
        sloped_count += 1

# Calculate the ratio of sloped to flat roofs
if flat_count > 0:  # Ensure there are flat roofs to avoid division by zero
    ratio_sloped_to_flat = sloped_count / flat_count
else:
    ratio_sloped_to_flat = None  # Handle case with no flat roofs

print(f"Number of flat roofs: {flat_count}")
print(f"Number of sloped roofs: {sloped_count}")
print(f"Ratio of sloped roofs to flat roofs: {ratio_sloped_to_flat}")



0: 544x640 21 flats, 25 slopeds, 1033.2ms
Speed: 8.8ms preprocess, 1033.2ms inference, 39.1ms postprocess per image at shape (1, 3, 544, 640)
Number of flat roofs: 21
Number of sloped roofs: 25
Ratio of sloped roofs to flat roofs: 1.1904761904761905
